In [ ]:
#%pip install duckdb fsspec gcsfs

In [2]:
import duckdb
from fsspec import filesystem
import os

In [ ]:
cn = duckdb.connect()
cn.register_filesystem(filesystem('gcs'))
cn.execute("INSTALL ducklake")

In [4]:
GCS_BUCKET = os.environ.get('GCS_BUCKET')
cn.execute(f"ATTACH 'ducklake:gcs_flock.ducklake' as gcs_flock (DATA_PATH 'gs://{GCS_BUCKET}/gcs_flock')")
cn.execute("USE gcs_flock")

In [10]:
sql = """
create or replace table order_header 
as 
select 1 as order_id, date('2023-01-01') as order_dt, 1001 as cust_id, 250.75 as ord_amount
union all select 2, date('2023-01-02'), 1002, 150.50
union all select 3, date('2023-01-03'), 1003, 300.00
"""
cn.execute(sql)

In [11]:
sql = """
update order_header
set ord_amount = ord_amount * 1.1
where order_id = 2
"""
cn.execute(sql)

In [12]:
cn.sql("from order_header").show()

┌──────────┬────────────┬─────────┬──────────────┐
│ order_id │  order_dt  │ cust_id │  ord_amount  │
│  int32   │    date    │  int32  │ decimal(5,2) │
├──────────┼────────────┼─────────┼──────────────┤
│        1 │ 2023-01-01 │    1001 │       250.75 │
│        3 │ 2023-01-03 │    1003 │       300.00 │
│        2 │ 2023-01-02 │    1002 │       165.55 │
└──────────┴────────────┴─────────┴──────────────┘

